# Search POI Hania

----
Libraries:

In [1]:
import re
import numpy as np
import pandas as pd
import json 
from batoomer.twitter_nodes.search_engine import GoogleSearchEngine
from googleapiclient.errors import HttpError
from tqdm.notebook import tqdm
import time

---
Load Google Credentials and initialize GoogleSearchEngine

In [5]:
keys = []
with open('../../../google_credentials.json', 'r') as f:
    keys = json.load(f)

se = GoogleSearchEngine(google_api_key=keys['api_key'], search_engine_id=keys['twitter_seID'])

---
## Dataset

In [13]:
data = pd.read_csv('../../../datasets/poi-hania/lista-poi-hania.csv', delimiter=';')
data.replace(np.nan, '', inplace=True)
data.head(5)

,ΔΗΜΟΣ,ΚΑΤΗΓΟΡΙΑ,ΑΝΤΙΚΕΙΜΕΝΟ,ΠΕΡΙΟΧΗ,ΣΗΜΕΙΟ ΕΝΔΙΑΦΕΡΟΝΤΟΣ
0,ΓΑΥΔΟΥ,ΙΣΤΟΡΙΑ,ΑΡΧΑΙΟΛΟΓΙΚΑ ΜΝΗΜΕΙΑ,ΓΑΥΔΟΥ,ΜΙΝΩΙΚΗ ΒΙΛΛΑ
1,ΓΑΥΔΟΥ,ΙΣΤΟΡΙΑ,ΑΡΧΑΙΟΛΟΓΙΚΑ ΜΝΗΜΕΙΑ,ΓΑΥΔΟΥ,ΡΩΜΑΪΚΟ ΕΛΑΙΟΤΡΙΒΕΙΟ
2,ΓΑΥΔΟΥ,ΙΣΤΟΡΙΑ,ΑΡΧΑΙΟΛΟΓΙΚΑ ΜΝΗΜΕΙΑ,ΓΑΥΔΟΥ,ΡΩΜΑΪΚΟ ΥΔΡΑΓΩΓΕΙΟ
3,ΓΑΥΔΟΥ,ΙΣΤΟΡΙΑ,ΑΡΧΑΙΟΛΟΓΙΚΑ ΜΝΗΜΕΙΑ,ΓΑΥΔΟΥ,ΔΥΟ ΜΙΝΩΪΚΟΙ ΤΑΦΟΙ
4,ΚΑΝΔΑΝΟΥ- ΣΕΛΙΝΟΥ,ΙΣΤΟΡΙΑ,ΑΡΧΑΙΟΛΟΓΙΚΑ ΜΝΗΜΕΙΑ,-,ΕΛΥΡΟΣ


In [18]:
data = data.sample(frac=1).reset_index(drop=True)

In [21]:
data = data.iloc[:100]

In [24]:
data.head()

,ΔΗΜΟΣ,ΚΑΤΗΓΟΡΙΑ,ΑΝΤΙΚΕΙΜΕΝΟ,ΠΕΡΙΟΧΗ,ΣΗΜΕΙΟ ΕΝΔΙΑΦΕΡΟΝΤΟΣ
0,ΑΠΟΚΟΡΩΝΟΥ,ΙΣΤΟΡΙΑ,ΕΚΚΛΗΣΙΕΣ,Εξώπολη,εκκλησία Παναγίας & Αγ. Δημητρίου
1,ΣΦΑΚΙΩΝ,ΦΥΣΙΚΟ ΠΕΡΙΒΑΛΛΟΝ,ΠΑΡΑΛΙΕΣ,ΧΩΡΑ ΣΦΑΚΙΩΝ,ΒΡΥΣΗ
2,ΚΑΝΔΑΝΟΥ- ΣΕΛΙΝΟΥ,ΙΣΤΟΡΙΑ,ΕΚΔΗΛΩΣΕΙΣ & ΓΙΟΡΤΕΣ,-,ΟΛΟΚΑΥΤΩΜΑ ΚΑΝΤΑΝΟΥ 1-3 ΙΟΥΝΙΟΥ
3,ΚΑΝΔΑΝΟΥ- ΣΕΛΙΝΟΥ,ΙΣΤΟΡΙΑ,ΕΚΔΗΛΩΣΕΙΣ & ΓΙΟΡΤΕΣ,-,ΚΑΡΝΑΒΑΛΙ ΠΑΛΑΙΟΧΩΡΑΣ
4,ΑΠΟΚΟΡΩΝΟΥ,ΙΣΤΟΡΙΑ,ΙΣΤΟΡΙΚΑ ΜΝΗΜΕΙΑ,Βάμος,Παρθεναγωγείο


---
## Search: ΔΗΜΟΣ + POI

In [31]:
queries = list((data['ΔΗΜΟΣ'] + " " + data['ΣΗΜΕΙΟ ΕΝΔΙΑΦΕΡΟΝΤΟΣ']).str.lower())
results = pd.DataFrame()

for query in tqdm(queries):
    try:
        se.search(query=query)
        result = se.get_results()
        results= results.append(result)
    except HttpError as err:
        if err.resp.status == 429:
            print('Api Limit Hit!')
            time.sleep(60 * 60 * 24)
            se.search(query=query)
            result = se.get_results()
            results= results.append(result)

  0%|          | 0/100 [00:00<?, ?it/s]

In [32]:
results

,Query,Result 1,Result 2,Result 3
0,αποκορωνου εκκλησία παναγίας & αγ. δημητρίου,NaN,NaN,NaN
0,σφακιων βρυση,NaN,NaN,NaN
0,κανδανου- σελινου ολοκαυτωμα καντανου 1-3 ιουνιου,NaN,NaN,NaN
0,κανδανου- σελινου καρναβαλι παλαιοχωρας,NaN,NaN,NaN
0,αποκορωνου παρθεναγωγείο,NaN,NaN,NaN
...,...,...,...,...
0,αποκορωνου παλιό ελαιοτριβείο,NaN,NaN,NaN
0,γαυδου κεδρές,NaN,NaN,NaN
0,κανδανου- σελινου καστρο φορτετζα παλαιοχωρας,NaN,NaN,NaN
0,γαυδου 38 μετοχια,NaN,NaN,NaN


In [33]:
results = (results
           .replace(np.nan, '')
           .rename(columns={'Query':'poi'})
           .reset_index()
           .drop('index', axis=1))
results.head(5)

,poi,Result 1,Result 2,Result 3
0,αποκορωνου εκκλησία παναγίας & αγ. δημητρίου,,,
1,σφακιων βρυση,,,
2,κανδανου- σελινου ολοκαυτωμα καντανου 1-3 ιουνιου,,,
3,κανδανου- σελινου καρναβαλι παλαιοχωρας,,,
4,αποκορωνου παρθεναγωγείο,,,


In [34]:
results.to_csv('Google-Search-poi-hania-test.csv', index=False)